In [1]:
#pandas and numpy for df manipulation
import pandas as pd
import numpy as np
import re

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#counter
from collections import Counter

#standard nlp libraries for text processing
import nltk

#gensim will not be used in this notebook, but it is a very popular and useful library
import gensim

#useful functions frequency vector functions 
from sklearn.feature_extraction.text import CountVectorizer #sklearn
from nltk.probability import FreqDist #nltk

#useful functions for tf-idf vectorization
from sklearn.feature_extraction.text import TfidfVectorizer #sklearn

#ngrams
from nltk.util import ngrams #nltk

In [2]:
path = 'C:\\Users\\ritaf\\Documents\\ESTUDOS\\NOVA IMS\\3º Ano\\Text Mining\\Projeto\\new_data\\'

# Read the CSV file into a DataFrame
data = pd.read_csv(path + 'text_train_btp.csv')


In [3]:
data.head()

,title,artist,features,lyrics,tag,title_token,lyrics_token,title_token_filtered,lyrics_token_filtered,title_filtered_string,lyrics_filtered_string,title_string_fdist,lyrics_string_fdist
0,walk away,tony molina,NaN,when you said you loved medid you mean it then...,rock,"['walk', 'away']","['when', 'you', 'said', 'you', 'loved', 'medid...","['walk', 'away']","['said', 'loved', 'medid', 'mean', 'thendid', ...",walk away,said loved medid mean thendid change mind one ...,walk away,said loved medid mean change mind one dayor pr...
1,gotta make it kid naruto rap,reece lett,slck,kid naruto raphookeverybody wants you to hurte...,rap,"['gotta', 'make', 'it', 'kid', 'naruto', 'rap']","['kid', 'naruto', 'raphookeverybody', 'wants',...","['gotta', 'make', 'kid', 'naruto', 'rap']","['kid', 'naruto', 'raphookeverybody', 'wants',...",gotta make kid naruto rap,kid naruto raphookeverybody wants hurteverybod...,make kid naruto rap,kid naruto wants hurteverybody wants cry face ...
2,this is what i asked for,elliot dnk,NaN,verse 1people tell me ive changedi find it har...,pop,"['this', 'is', 'what', 'i', 'asked', 'for']","['verse', '1people', 'tell', 'me', 'ive', 'cha...",['asked'],"['verse', '1people', 'tell', 'ive', 'changedi'...",asked,verse 1people tell ive changedi find hard expl...,asked,verse 1people tell ive changedi find hard expl...
3,stealing hearts,katie armiger,NaN,youve been warned about medont try to get too ...,pop,"['stealing', 'hearts']","['youve', 'been', 'warned', 'about', 'medont',...","['stealing', 'hearts']","['youve', 'warned', 'medont', 'try', 'get', 'c...",stealing hearts,youve warned medont try get closei know want c...,stealing hearts,youve warned medont try get closei know want c...
4,get ready,john campbell munro,NaN,verse 1i can see the end is coming but ive nev...,country,"['get', 'ready']","['verse', '1i', 'can', 'see', 'the', 'end', 'i...","['get', 'ready']","['verse', '1i', 'see', 'end', 'coming', 'ive',...",get ready,verse see end coming ive never liked oddsi nee...,get ready,verse see end coming ive never liked oddsi nee...


In [9]:
data

,title,artist,features,lyrics,tag,title_token,lyrics_token,title_token_filtered,lyrics_token_filtered,title_filtered_string,lyrics_filtered_string,title_string_fdist,lyrics_string_fdist
0,walk away,tony molina,NaN,when you said you loved medid you mean it then...,rock,"['walk', 'away']","['when', 'you', 'said', 'you', 'loved', 'medid...","['walk', 'away']","['said', 'loved', 'medid', 'mean', 'thendid', ...",walk away,said loved medid mean thendid change mind one ...,walk away,said loved medid mean change mind one dayor pr...
1,gotta make it kid naruto rap,reece lett,slck,kid naruto raphookeverybody wants you to hurte...,rap,"['gotta', 'make', 'it', 'kid', 'naruto', 'rap']","['kid', 'naruto', 'raphookeverybody', 'wants',...","['gotta', 'make', 'kid', 'naruto', 'rap']","['kid', 'naruto', 'raphookeverybody', 'wants',...",gotta make kid naruto rap,kid naruto raphookeverybody wants hurteverybod...,make kid naruto rap,kid naruto wants hurteverybody wants cry face ...
2,this is what i asked for,elliot dnk,NaN,verse 1people tell me ive changedi find it har...,pop,"['this', 'is', 'what', 'i', 'asked', 'for']","['verse', '1people', 'tell', 'me', 'ive', 'cha...",['asked'],"['verse', '1people', 'tell', 'ive', 'changedi'...",asked,verse 1people tell ive changedi find hard expl...,asked,verse 1people tell ive changedi find hard expl...
3,stealing hearts,katie armiger,NaN,youve been warned about medont try to get too ...,pop,"['stealing', 'hearts']","['youve', 'been', 'warned', 'about', 'medont',...","['stealing', 'hearts']","['youve', 'warned', 'medont', 'try', 'get', 'c...",stealing hearts,youve warned medont try get closei know want c...,stealing hearts,youve warned medont try get closei know want c...
4,get ready,john campbell munro,NaN,verse 1i can see the end is coming but ive nev...,country,"['get', 'ready']","['verse', '1i', 'can', 'see', 'the', 'end', 'i...","['get', 'ready']","['verse', '1i', 'see', 'end', 'coming', 'ive',...",get ready,verse see end coming ive never liked oddsi nee...,get ready,verse see end coming ive never liked oddsi nee...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134962,manhattan,tijuana sweetheart,NaN,if i knew when i was young that id be olderthe...,pop,['manhattan'],"['if', 'i', 'knew', 'when', 'i', 'was', 'young...",['manhattan'],"['knew', 'young', 'id', 'olderthere', 'many', ...",manhattan,knew young olderthere many things wouldve told...,manhattan,knew young many things wouldve told youbut did...
134963,belly shit,lil gotit,lil troup,intro lil gotitcashwahwahwahwahwahwaheverythin...,rap,"['belly', 'shit']","['intro', 'lil', 'gotitcashwahwahwahwahwahwahe...","['belly', 'shit']","['intro', 'lil', 'gotitcashwahwahwahwahwahwahe...",belly shit,intro lil gotitcashwahwahwahwahwahwaheverythin...,belly shit,intro lil chrome futurechorus lil troup lil dr...
134964,the four loves,heath mcnease,NaN,verse 1i was born inside a homethe youngest of...,rock,"['the', 'four', 'loves']","['verse', '1i', 'was', 'born', 'inside', 'a', ...","['four', 'loves']","['verse', '1i', 'born', 'inside', 'homethe', '...",four loves,verse born inside homethe youngest threeboys f...,four loves,verse born inside homethe youngest fought dadd...
134965,ball and chain,transmatic,NaN,rollin by the pool the falling stars are not c...,pop,"['ball', 'and', 'chain']","['rollin', 'by', 'the', 'pool', 'the', 'fallin...","['ball', 'chain']","['rollin', 'pool', 'falling', 'stars', 'condes...",ball chain,rollin pool falling stars condescendingit twin...,ball chain,rollin pool falling stars twins nothing nothin...


In [4]:
lyr = data['lyrics'].to_list()
#lyr_f = data['lyrics_filtered_string'].to_list()
#lyr_fd = data['lyrics_string_fdist'].to_list()

lyr

['when you said you loved medid you mean it thendid you change your mind one dayor was it all pretendcause its over nowsee you walk awaysee you walk awayseems like nothings gonna change',
 'kid naruto raphookeverybody wants you to hurteverybody wants you to cry with your face in the dirtgotta put in time gotta workeverybodys praying that i fallbut i gotta make it workeverybody wants you to hurteverybody wants you to cry with your face in the dirtgotta put in time gotta workeverybodys praying that i fallbut i gotta make itversehoping it might work out one daywrongdo it yourself or you might as well fallgotta do the grindif you never wanna fallbetter aim for the top or you never won at allas a kid i had nobodygrew up parentlesshad to go and be funnyhad to hide the pain some wayon my own paththats my ninja wayheaded for the topon a day to daythey treat me like outcastgot stay out the waygotta hide emotionsgotta hid the painthis demon fox insidekeeps on driving m insanehookeverybody wants 

In [5]:
def tokenized_frequency_vector(list_of_strings):
    '''Takes as input a list of strings of text and returns a list of dictionaries representing the frequency of each tokenized word'''

    # Create a tokenizer that only captures letters and numbers
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

    # Initialize an empty list to store the frequency vectors
    frequency_vectors = []

    # Process each string in the input list
    for text in list_of_strings:
    
        # Tokenize the text data - converted to string to avoid issues with numbers or other data types
        tokens = tokenizer.tokenize(str(text))

        # Convert the tokens to lowercase
        tokens = [token.lower() for token in tokens]

        # Remove stop words
        #1. Create a set of stop words
        stop_words = set(nltk.corpus.stopwords.words('english')) #CODE HERE
    
        #remove stop words from the tokenized text
        tokens = [token for token in tokens if token not in stop_words] #CODE HERE

        # Create a frequency distribution for the tokenized text data
        fdist = FreqDist(tokens)

        # Append the frequency vector to the list of frequency vectors
        frequency_vectors.append(dict(fdist))

    return frequency_vectors

In [7]:
messages_freq_vectors = tokenized_frequency_vector(lyr)
messages_freq_vectors

[{'said': 1,
  'loved': 1,
  'medid': 1,
  'mean': 1,
  'thendid': 1,
  'change': 2,
  'mind': 1,
  'one': 1,
  'dayor': 1,
  'pretendcause': 1,
  'nowsee': 1,
  'walk': 2,
  'awaysee': 1,
  'awayseems': 1,
  'like': 1,
  'nothings': 1,
  'gonna': 1},
 {'kid': 2,
  'naruto': 1,
  'raphookeverybody': 1,
  'wants': 14,
  'hurteverybody': 7,
  'cry': 7,
  'face': 7,
  'dirtgotta': 7,
  'put': 7,
  'time': 7,
  'gotta': 14,
  'workeverybodys': 7,
  'praying': 7,
  'fallbut': 7,
  'make': 8,
  'workeverybody': 4,
  'itversehoping': 1,
  'might': 2,
  'work': 1,
  'one': 1,
  'daywrongdo': 1,
  'well': 1,
  'fallgotta': 1,
  'grindif': 1,
  'never': 2,
  'wanna': 1,
  'fallbetter': 1,
  'aim': 1,
  'top': 1,
  'allas': 1,
  'nobodygrew': 1,
  'parentlesshad': 1,
  'go': 1,
  'funnyhad': 1,
  'hide': 2,
  'pain': 1,
  'wayon': 1,
  'paththats': 1,
  'ninja': 1,
  'wayheaded': 1,
  'topon': 1,
  'day': 1,
  'daythey': 1,
  'treat': 1,
  'like': 1,
  'outcastgot': 1,
  'stay': 1,
  'waygotta': 

In [8]:
messages_freq_df_1 = pd.DataFrame(messages_freq_vectors)

# Sort the columns by their sums in descending order
sorted_cols = messages_freq_df_1.sum().sort_values(ascending=False).index

# Reorder the columns of the DataFrame
messages_freq_df_1 = messages_freq_df_1[sorted_cols]

#this creates a sparse matrix
messages_freq_df_1

MemoryError: Unable to allocate 1.99 TiB for an array with shape (134967, 2028421) and data type object